# This file is for testing the simulator with real data

First, let's initialize the starting positions and the desired end positions. 
For this, the Hosizons System data service is used.


In [36]:
import sys
sys.path.append('..')
import importlib
import tensorflow as tf
import Data.Horizons as horizon
import rebound
import numpy as np
import math
importlib.reload(horizon)

def initValues():
    tau = 0.1

    r0, v0, r1, v1, m = horizon.get_values_for_2024_02_28_and2024_02_09_fast()

    n = len(r0)
    
    return tau, n, m, r0, v0, r1, v1

### Simulation of one day with the current n-body simulator:

In [37]:
import numpy as np
import Learning.find_minimum as fm

tau, n, m, r0, v0, r1, v1 = initValues()

x, y = fm.execute_x_times_do_step_normal(tau, n, m, r0, v0, 10)
print("How it should be:", f"r1: {r1}",
      "Simulator output:", f"r1: {x}", sep="\n")
print("How it should be:", f"v1:{v1}",
      "Simulator output:", f"v1:{y}", sep="\n")

print("differences in positions: ", np.abs(np.array(r1) - np.array(x)), "\n differences in velocities: ",
      np.abs(np.array(v1) - np.array(y)))  # The differences don't get smaller with smapper tau

59
How it should be:
r1: [[-0.007668292490862695, -0.003308848670945613, 0.0002072977430167829], [0.349629484725005, -0.1222013394405457, -0.04228125160049356], [0.1229828037999395, -0.7187514452301442, -0.01715689882307804], [-0.9358499829949305, 0.3427213062305836, 0.0001941610404147327], [0.5398647146263628, -1.306853952358735, -0.04054163395824471], [3.148436290823115, 3.869840604270508, -0.08649380944161839], [9.090527183729291, -3.413539422101748, -0.302585712868104], [12.07942658341018, 15.43066830062747, -0.09918161542644022], [29.84432715520231, -1.607806805652261, -0.654683684008933], [-0.5520602121187203, -2.757908443003486, 0.01333104527791402], [17.36769428034018, -30.29046982875162, -1.782512506151329], [85.52162430688986, 38.87887242352961, -18.12815672916991], [-46.09810616345356, -7.613395917267925, 24.34872599219304], [-37.93905342969168, -22.42315657020927, 23.62826288522441], [-0.9382207656515614, 0.3414564700848016, 0.0001414843519881807], [0.5398081818461327, -1.3

## Results:
The simulator doesn't perform well on real data and doesn't predict the positions and velocities of celestial objects after one day

# Rebound:

Let's try a differen simulator to ensure the problem is with the simulator. This simulator is called Rebound.


In [33]:
# init values
tau, n, m, r0, v0, r1, v1 = initValues()

# Create a new simulation
sim = rebound.Simulation()

sim.units = ('day', 'AU', 'Msun')

# Add the Sun and planets with the given initial conditions
for i in range(len(m)):
    sim.add(m=m[i], x=r0[i][0], y=r0[i][1], z=r0[i]
            [2], vx=v0[i][0], vy=v0[i][1], vz=v0[i][2])

# Set the integrator to a high-accuracy one
sim.integrator = "ias15"

print(sim.G)

# Set the time step
sim.dt = 1/4800

# Integrate the simulation for one day
sim.integrate(1)

# Extract the final positions and velocities
final_positions = []
final_velocities = []
for particle in sim.particles:
    final_positions.append([particle.x, particle.y, particle.z])
    final_velocities.append([particle.vx, particle.vy, particle.vz])

# Convert to numpy arrays for easier comparison
final_positions = np.array(final_positions)
final_velocities = np.array(final_velocities)

# Print the final positions and velocities for comparison
print("Final Positions (REBOUND):")
print(final_positions)
print("Final Velocities (REBOUND):")
print(final_velocities)

print("\nExpected Final Positions (NASA Horizons):")
print(np.array(r1))
print("Expected Final Velocities (NASA Horizons):")
print(np.array(v1))

# Calculate and print the differences
position_differences = final_positions - np.array(r1)
velocity_differences = final_velocities - np.array(v1)
print("\nPosition Differences:")
print(position_differences)
print("Velocity Differences:")
print(velocity_differences)

132712440041.93938
0.00029591220828559104
Final Positions (REBOUND):
[[-7.66829249e-03 -3.30884867e-03  2.07297743e-04]
 [ 3.49629485e-01 -1.22201339e-01 -4.22812516e-02]
 [ 1.22982804e-01 -7.18751445e-01 -1.71568988e-02]
 [-9.35849983e-01  3.42721306e-01  1.94161041e-04]
 [ 5.39864715e-01 -1.30685395e+00 -4.05416340e-02]
 [ 3.14843629e+00  3.86984060e+00 -8.64938094e-02]
 [ 9.09052718e+00 -3.41353942e+00 -3.02585713e-01]
 [ 1.20794266e+01  1.54306683e+01 -9.91816154e-02]
 [ 2.98443273e+01 -1.60780686e+00 -6.54683877e-01]
 [-5.52060212e-01 -2.75790844e+00  1.33310453e-02]
 [ 1.73676943e+01 -3.02904698e+01 -1.78251251e+00]
 [ 8.55216243e+01  3.88788724e+01 -1.81281567e+01]
 [-4.60981062e+01 -7.61339592e+00  2.43487260e+01]
 [-3.79390534e+01 -2.24231566e+01  2.36282629e+01]
 [-9.38220768e-01  3.41456469e-01  1.41484325e-04]
 [ 5.39808099e-01 -1.30686156e+00 -4.05146919e-02]
 [ 5.39957999e-01 -1.30696956e+00 -4.05919635e-02]
 [ 3.15095504e+00  3.87108898e+00 -8.64131126e-02]
 [ 3.14412532

## Results:
It seems like there is some other problem, that I don't understand. This simulater also produces wrong results. Let's compare the results from Rebound with our simulator

In [34]:
print("\nExpected Final Positions (NASA Horizons):")
print(np.array(r1))
print("Final positions Rebound:")
print(final_positions)
print("Final positions Network:")
print(np.array(x))
print("Differences between Rebound and the Network:")
print(np.array(x) - final_positions)

print("Expected Final Velocities (NASA Horizons):")
print(np.array(v1))
print("Final Velocities Rebound:")
print(final_velocities)
print("Final Velocities Network:")
print(np.array(y))
print("Differences between Rebound and the Network:")
print(np.array(y) - final_velocities)


Expected Final Positions (NASA Horizons):
[[-7.66829249e-03 -3.30884867e-03  2.07297743e-04]
 [ 3.49629485e-01 -1.22201339e-01 -4.22812516e-02]
 [ 1.22982804e-01 -7.18751445e-01 -1.71568988e-02]
 [-9.35849983e-01  3.42721306e-01  1.94161040e-04]
 [ 5.39864715e-01 -1.30685395e+00 -4.05416340e-02]
 [ 3.14843629e+00  3.86984060e+00 -8.64938094e-02]
 [ 9.09052718e+00 -3.41353942e+00 -3.02585713e-01]
 [ 1.20794266e+01  1.54306683e+01 -9.91816154e-02]
 [ 2.98443272e+01 -1.60780681e+00 -6.54683684e-01]
 [-5.52060212e-01 -2.75790844e+00  1.33310453e-02]
 [ 1.73676943e+01 -3.02904698e+01 -1.78251251e+00]
 [ 8.55216243e+01  3.88788724e+01 -1.81281567e+01]
 [-4.60981062e+01 -7.61339592e+00  2.43487260e+01]
 [-3.79390534e+01 -2.24231566e+01  2.36282629e+01]
 [-9.38220766e-01  3.41456470e-01  1.41484352e-04]
 [ 5.39808182e-01 -1.30686244e+00 -4.05148098e-02]
 [ 5.39958130e-01 -1.30696944e+00 -4.05920105e-02]
 [ 3.15094506e+00  3.87110082e+00 -8.64128455e-02]
 [ 3.14412739e+00  3.86875519e+00 -8.66

For better visualisation, here it is with columns directly underneath each other:


In [35]:
planets = ['Sun', 'Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune', 'Ceres', 'Pluto', 'Eris', 'Makemake', 'Haumea', 'Moon', 'Phobos', 'Deimos', 'Io', 'Europa', 'Ganymede', 'Callisto', 'Amalthea', 'Himalia', 'Thebe', 'Adrastea', 'Metis', 'Mimas', 'Enceladus', 'Tethys',
                                                                                                          'Dione', 'Rhea', 'Titan', 'Hyperion', 'Iapetus', 'Phoebe', 'Janus', 'Epimetheus', 'Helene', 'Atlas', 'Prometheus', 'Pandora', 'Pan', 'Ariel', 'Umbriel', 'Titania', 'Oberon', 'Miranda', 'Triton', 'Naiad', 'Thalassa', 'Despina', 'Galatea', 'Larissa', 'Proteus', 'Charon', 'Nix', 'Hydra', 'Kerberos', 'Styx']
for i in range(len(planets)):
    print(f"\n {planets[i]} \n")
    print("Expected Final Position:", np.array(r1)[i])
    print("Final positions Rebound:", final_positions[i])
    print("Final positions Network:", np.array(x)[i])
    print("Differences between Rebound and the Network:")
    print(np.array(x)[i] - final_positions[i])

    print("Expected Final Velocity:", np.array(v1)[i])
    print("Final  Velocity Rebound:", final_velocities[i])
    print("Final  Velocity Network:", np.array(y)[i])
    print("Differences between Rebound and the Network:")
    print(np.array(y)[i] - final_velocities[i])


 Sun 

Expected Final Position: [-0.00766829 -0.00330885  0.0002073 ]
Final positions Rebound: [-0.00766829 -0.00330885  0.0002073 ]
Final positions Network: [-0.00766829 -0.00330885  0.0002073 ]
Differences between Rebound and the Network:
[-5.65433117e-14 -5.33475174e-14  2.16585647e-15]
Expected Final Velocity: [ 5.29181283e-06 -6.60543214e-06 -5.80723869e-08]
Final  Velocity Rebound: [ 5.29181248e-06 -6.60543247e-06 -5.80723759e-08]
Final  Velocity Network: [ 5.29181237e-06 -6.60543258e-06 -5.80723715e-08]
Differences between Rebound and the Network:
[-1.13177919e-13 -1.06747541e-13  4.33709392e-15]

 Mercury 

Expected Final Position: [ 0.34962948 -0.12220134 -0.04228125]
Final positions Rebound: [ 0.34962948 -0.12220134 -0.04228125]
Final positions Network: [ 0.3496295  -0.12220134 -0.04228125]
Differences between Rebound and the Network:
[ 1.37035172e-08 -5.30597728e-09 -1.69053951e-09]
Expected Final Velocity: [0.00341625 0.0279539  0.00197204]
Final  Velocity Rebound: [0.0034